In [1]:
!pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu110/torch-1.7.0%2Bcu110-cp38-cp38-win_amd64.whl (2046.8 MB)
  Using cached https://download.pytorch.org/whl/cu110/torchvision-0.8.1%2Bcu110-cp38-cp38-win_amd64.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/torchaudio-0.7.0-cp38-none-win_amd64.whl (103 kB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.1
    Uninstalling torchvision-0.9.1:
      Successfully uninstalled torchvision-0.9.1


### Построение нейросети, распознающей рукописные цифры

### Построение нейросети

In [1]:
import numpy as np
import torch
import seaborn as sns
from matplotlib import pyplot as plt

# import standard PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import pandas as pd

In [2]:
torch.manual_seed(1)
np.random.seed(1)

In [3]:
train = torch.load('./data/train.pt')
test = torch.load('./data/test.pt')
valid = torch.load('./data/valid.pt')

In [4]:
batch_size = 1000
train_loader = DataLoader(dataset=train, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset=test, batch_size=1000, shuffle=False)
valid_loader = DataLoader(dataset=valid, batch_size=1000, shuffle=False)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
class ConvNet(nn.Module):
    def __init__(self):

        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
             nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
             nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out1 = nn.Dropout(0.4)
        self.bn1 = nn.BatchNorm1d(7 * 7 * 64)
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.drop_out2 = nn.Dropout(0.4)
        self.bn2 = nn.BatchNorm1d(1000)
        self.fc2 = nn.Linear(1000, 500)
        self.drop_out3 = nn.Dropout(0.4)
        self.bn3 = nn.BatchNorm1d(500)
        self.fc3 = nn.Linear(500, 10)
      
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out1(out)
        out = self.bn1(out)
        out = self.fc1(out)
        out = self.drop_out2(out)
        out = self.bn2(out)
        out = self.fc2(out)
        out = self.drop_out3(out)
        out = self.bn3(out)
        out = self.fc3(out)
        return out

In [7]:
num_epochs = 1000
num_classes = 10
learning_rate = 1e-3

In [8]:
from  torch.optim.lr_scheduler import StepLR

In [9]:
model = ConvNet()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=10, gamma=0.7)

In [10]:
# Train the model
total_step = len(train_loader)
train_losses = []
test_losses = []
eps = 2e-4
eps_counter = 3
for epoch in range(num_epochs):
    train_loss = []
    for i, (images, labels) in enumerate(train_loader):
        images.size()
        # Run the forward pass
        images = images.float().to(device)
        labels = labels.to(device)
        outputs = model(images.unsqueeze(1))
        loss = criterion(outputs, labels)
        train_loss.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    with torch.no_grad():
        images = test.tensors[0].float().to(device)
        labels = test.tensors[1].to(device)
        output = model(images.unsqueeze(1))  
        loss = criterion(output, labels)
        test_losses.append(loss.cpu().detach().numpy())
            
    train_losses.append(np.mean(train_loss)) 
    print('Epoch [{}/{}], Train Loss(mean): {:.4f}, Test Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, np.mean(train_loss),
                          test_losses[epoch]))
    #early stopper
    if len(test_losses) >= 2 and np.abs(test_losses[-1] - test_losses[-2]) < eps:
          eps_counter = eps_counter - 1
    if eps_counter < 1:
        print('Finished')
        break
print('Finished')

Epoch [1/1000], Train Loss(mean): 0.2334, Test Loss: 0.0908
Epoch [2/1000], Train Loss(mean): 0.0813, Test Loss: 0.0776
Epoch [3/1000], Train Loss(mean): 0.0746, Test Loss: 0.0765
Epoch [4/1000], Train Loss(mean): 0.0715, Test Loss: 0.0744
Epoch [5/1000], Train Loss(mean): 0.0718, Test Loss: 0.0739
Epoch [6/1000], Train Loss(mean): 0.0723, Test Loss: 0.0778
Epoch [7/1000], Train Loss(mean): 0.0708, Test Loss: 0.0743
Epoch [8/1000], Train Loss(mean): 0.0726, Test Loss: 0.0770
Epoch [9/1000], Train Loss(mean): 0.0729, Test Loss: 0.0758
Epoch [10/1000], Train Loss(mean): 0.0723, Test Loss: 0.0745
Epoch [11/1000], Train Loss(mean): 0.0716, Test Loss: 0.0775
Epoch [12/1000], Train Loss(mean): 0.0720, Test Loss: 0.0748
Epoch [13/1000], Train Loss(mean): 0.0714, Test Loss: 0.0783
Epoch [14/1000], Train Loss(mean): 0.0719, Test Loss: 0.0746
Epoch [15/1000], Train Loss(mean): 0.0721, Test Loss: 0.0750
Epoch [16/1000], Train Loss(mean): 0.0723, Test Loss: 0.0745
Epoch [17/1000], Train Loss(mean)

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)
plt.show()

### Отложенная выборка

In [15]:
model.eval()
with torch.no_grad():
    images = valid.tensors[0].float().to(device)
    labels = valid.tensors[1].to(device)
    output = model(images.unsqueeze(1))  
    loss = criterion(output, labels)
    print('Final validation Loss {}'.format(loss))

10000
Final validation Loss 0.05595099553465843


In [28]:
valid_acc = torch.sum(torch.argmax(output, dim=1) == labels) / valid.tensors[0].size()[0]

In [29]:
print('Final validation Accuracy {} %'.format(valid_acc*100))

Final validation Accuracy 98.55999755859375 %


In [30]:
#todo: на каких цифрах чаще всего ошибается модел

### Сохранение модели

In [31]:
torch.save(model.state_dict(), 'mnist-cnn-model.pt')